In [6]:
import pandas as pd
import glob
import re
import xml.etree.ElementTree as et

Code given and modified

In [7]:
data_annotation = pd.DataFrame(columns = ['Sample_id', 'Cns_subregion', 'Sample_group', 'Subject_id']) # initialisation of the dataframe
xtree = et.parse('./Data/GSE124439_family.xml') # create a variable containing the xml in a tree shape
xroot = xtree.getroot() # get the root of the tree to start the exploration of the tree/xml
# for each element named "sample" that can be found from the root
for child in xroot.iter("{http://www.ncbi.nlm.nih.gov/geo/info/MINiML}Sample"):
    temp_sample_id = child.attrib['iid'] # the attribut of this node contains the sample id ()
    # for each element named "Characteristics" that can be found from the current sample
    for child2 in child.iter("{http://www.ncbi.nlm.nih.gov/geo/info/MINiML}Characteristics"):
        if(child2.attrib["tag"] == "cns subregion"):
            temp_cns_subregion = child2.text.replace('\n', '')
        if(child2.attrib["tag"] == "sample group"):
            temp_sample_group = child2.text.replace('\n', '')
        if(child2.attrib["tag"] == "subject id"):
            temp_subject_id = child2.text.replace('\n', '')
    temp_df = pd.DataFrame({'Sample_id': [temp_sample_id.strip()], 'Cns_subregion': [temp_cns_subregion.strip()], 'Sample_group': [temp_sample_group.strip()], 'Subject_id': [temp_subject_id.strip()]})
    data_annotation = pd.concat([data_annotation, temp_df])

In [8]:
data_annotation

,Sample_id,Cns_subregion,Sample_group,Subject_id
0,GSM3533230,Frontal Cortex,ALS Spectrum MND,NEUEL133AK6
0,GSM3533231,Frontal Cortex,ALS Spectrum MND,NEUYV496XLP
0,GSM3533232,Frontal Cortex,ALS Spectrum MND,NEUZU200WEQ
0,GSM3533233,Frontal Cortex,ALS Spectrum MND,NEUME287RK2
0,GSM3533234,Frontal Cortex,ALS Spectrum MND,NEUEC006FND
...,...,...,...,...
0,GSM3533401,Frontal Cortex,ALS Spectrum MND,NEUMY028JH4
0,GSM3533402,Frontal Cortex,ALS Spectrum MND,NEUXR145UBL
0,GSM3533403,Frontal Cortex,ALS Spectrum MND,NEUKN209FNW
0,GSM3533404,Frontal Cortex,ALS Spectrum MND,NEUUF289NRL


In [9]:
data_annotation['Cns_subregion'].unique()

array(['Frontal Cortex', 'Motor Cortex (Medial)',
       'Motor Cortex (Lateral)', 'Motor Cortex'], dtype=object)

In [10]:
data_annotation['Sample_group'].unique()

array(['ALS Spectrum MND', 'Non-Neurological Control',
       'Other Neurological Disorders'], dtype=object)

In [11]:
len(data_annotation['Subject_id'].unique())

95

Code given :

In [12]:
import pandas as pd
import glob
import re

path = "./Data" # the path of the data

pdList = [] # variable to temporary store all dataframes (one for each txt file)
# For all txt file
for fname in glob.glob(path+"/*.txt"):
    df = pd.read_table(fname) # put the file in a dataframe
    sample_name = re.search("GSM\d+", fname).group() # search the name of the sample in the file name
    df.rename(index= df["gene/TE"], inplace=True) # rename the index (=rows) using the column containing the gene name
    df.drop(columns=df.columns[0], axis=1, inplace=True) # drop the first column containing the gene name, no more need
    df.rename(columns={ df.columns[0]: sample_name }, inplace = True) # rename the column (there is only one column at this step) using the sample name
    pdList.append(df) # add the current dataframe in the list

In [13]:
pdList

[                        GSM3533340
 A1BG                            48
 A1BG-AS1                       114
 A1CF                             0
 A2M                            572
 A2M-AS1                         68
 ...                            ...
 X8_LINE:CR1:LINE               283
 X9_LINE:L1:LINE                 94
 Zaphod2:hAT-Tip100:DNA         277
 Zaphod3:hAT-Tip100:DNA         961
 Zaphod:hAT-Tip100:DNA         1153
 
 [28953 rows x 1 columns],
                         GSM3533293
 A1BG                            38
 A1BG-AS1                       166
 A1CF                             0
 A2M                           5970
 A2M-AS1                        190
 ...                            ...
 X8_LINE:CR1:LINE               466
 X9_LINE:L1:LINE                129
 Zaphod2:hAT-Tip100:DNA         549
 Zaphod3:hAT-Tip100:DNA        1618
 Zaphod:hAT-Tip100:DNA         2049
 
 [28953 rows x 1 columns],
                         GSM3533379
 A1BG                            50
 A1BG-

In [14]:
data_matrix = pd.concat(pdList, axis=1) # concat all dataframe in 1 dataframe
data_matrix = data_matrix.transpose() # transpose the dataframe to get a more standard shape (samples x variables)

In [ ]:
data_matrix

,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,A3GALT2,A4GALT,A4GNT,...,X6B_LINE:CR1:LINE,X7A_LINE:CR1:LINE,X7B_LINE:CR1:LINE,X7C_LINE:CR1:LINE,X7D_LINE:CR1:LINE,X8_LINE:CR1:LINE,X9_LINE:L1:LINE,Zaphod2:hAT-Tip100:DNA,Zaphod3:hAT-Tip100:DNA,Zaphod:hAT-Tip100:DNA
GSM3533340,48,114,0,572,68,116,1,1,20,1,...,541,1211,1291,612,73,283,94,277,961,1153
GSM3533293,38,166,0,5970,190,153,0,0,129,1,...,980,1951,1968,976,101,466,129,549,1618,2049
GSM3533379,50,101,0,2316,85,183,7,3,23,0,...,731,1650,1596,837,90,407,104,368,1550,1666
GSM3533365,32,91,0,2537,87,84,2,1,42,1,...,637,1505,1384,724,90,358,65,348,1408,1409
GSM3533307,18,69,0,946,55,74,0,4,4,2,...,568,1401,1148,703,56,410,81,334,1213,1210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM3533332,41,60,0,1233,116,153,4,0,8,0,...,758,1844,1730,891,79,435,105,403,1251,1453
GSM3533295,39,78,0,1381,80,130,2,5,10,1,...,739,1478,1416,737,79,414,94,329,1186,1438
GSM3533368,16,98,0,1291,85,152,0,2,20,1,...,866,1938,1823,1075,91,553,130,448,1787,1824
GSM3533235,15,39,0,957,74,48,1,2,21,0,...,446,1066,1022,635,63,267,65,256,866,977


In [48]:
list_genes = list(data_matrix.columns.values)
list_genes

['A1BG',
 'A1BG-AS1',
 'A1CF',
 'A2M',
 'A2M-AS1',
 'A2ML1',
 'A2MP1',
 'A3GALT2',
 'A4GALT',
 'A4GNT',
 'AA06',
 'AAAS',
 'AACS',
 'AACSP1',
 'AADAC',
 'AADACL2',
 'AADACL2-AS1',
 'AADACL3',
 'AADACL4',
 'AADACP1',
 'AADAT',
 'AAED1',
 'AAGAB',
 'AAK1',
 'AAMDC',
 'AAMP',
 'AANAT',
 'AAR2',
 'AARD',
 'AARS',
 'AARS2',
 'AARSD1',
 'AASDH',
 'AASDHPPT',
 'AASS',
 'AATBC',
 'AATF',
 'AATK',
 'ABALON',
 'ABAT',
 'ABCA1',
 'ABCA10',
 'ABCA11P',
 'ABCA12',
 'ABCA13',
 'ABCA17P',
 'ABCA2',
 'ABCA3',
 'ABCA4',
 'ABCA5',
 'ABCA6',
 'ABCA7',
 'ABCA8',
 'ABCA9',
 'ABCA9-AS1',
 'ABCB1',
 'ABCB10',
 'ABCB11',
 'ABCB4',
 'ABCB5',
 'ABCB6',
 'ABCB7',
 'ABCB8',
 'ABCB9',
 'ABCC1',
 'ABCC10',
 'ABCC11',
 'ABCC12',
 'ABCC13',
 'ABCC2',
 'ABCC3',
 'ABCC4',
 'ABCC5',
 'ABCC5-AS1',
 'ABCC6',
 'ABCC6P1',
 'ABCC6P2',
 'ABCC8',
 'ABCC9',
 'ABCD1',
 'ABCD2',
 'ABCD3',
 'ABCD4',
 'ABCE1',
 'ABCF1',
 'ABCF2',
 'ABCF3',
 'ABCG1',
 'ABCG2',
 'ABCG4',
 'ABCG5',
 'ABCG8',
 'ABHD1',
 'ABHD10',
 'ABHD11',
 'ABHD11-AS

Merging the dataframe

In [18]:
data_frame_merged = data_annotation.merge(data_matrix, right_index=True, left_on="Sample_id")

In [19]:
data_frame_merged

,Sample_id,Cns_subregion,Sample_group,Subject_id,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,...,X6B_LINE:CR1:LINE,X7A_LINE:CR1:LINE,X7B_LINE:CR1:LINE,X7C_LINE:CR1:LINE,X7D_LINE:CR1:LINE,X8_LINE:CR1:LINE,X9_LINE:L1:LINE,Zaphod2:hAT-Tip100:DNA,Zaphod3:hAT-Tip100:DNA,Zaphod:hAT-Tip100:DNA
0,GSM3533230,Frontal Cortex,ALS Spectrum MND,NEUEL133AK6,42,149,0,3327,101,78,...,986,2218,2096,1120,84,556,134,570,2045,2446
0,GSM3533231,Frontal Cortex,ALS Spectrum MND,NEUYV496XLP,25,107,2,1787,153,109,...,1216,2373,2001,1144,118,582,153,552,2057,2279
0,GSM3533232,Frontal Cortex,ALS Spectrum MND,NEUZU200WEQ,20,95,0,1830,121,127,...,905,2018,1815,1011,84,571,126,479,1659,1778
0,GSM3533233,Frontal Cortex,ALS Spectrum MND,NEUME287RK2,22,101,0,2131,142,143,...,935,2278,2032,1198,97,562,132,540,2032,2135
0,GSM3533234,Frontal Cortex,ALS Spectrum MND,NEUEC006FND,22,102,0,1706,83,178,...,966,2295,1993,1152,96,582,124,539,1742,1939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,GSM3533401,Frontal Cortex,ALS Spectrum MND,NEUMY028JH4,23,113,0,2130,72,71,...,553,1243,1089,658,72,320,73,311,1102,1169
0,GSM3533402,Frontal Cortex,ALS Spectrum MND,NEUXR145UBL,134,300,0,6077,171,42,...,719,1988,1691,966,119,460,109,399,1619,1724
0,GSM3533403,Frontal Cortex,ALS Spectrum MND,NEUKN209FNW,64,290,0,4752,118,174,...,623,1466,1246,726,61,358,107,347,1284,1469
0,GSM3533404,Frontal Cortex,ALS Spectrum MND,NEUUF289NRL,124,332,0,3516,141,173,...,743,1846,1680,936,110,479,120,503,1646,1839


In [43]:
data_frame_merged = data_frame_merged.set_index('Sample_id')

In [55]:
class ALS_RNAseq:
    def __init__(self):
        self.__data_matrix = data_frame_merged
    
    def get_data_matrix(self):
        return self.__data_matrix
    
    def get_data_genes(self) :
        return self.get_data_matrix()[list_genes]
        
    def __str__(self):
        return "Data Frame of RNA seq samples"

In [56]:
c_data = ALS_RNAseq()

In [57]:
c_data.get_data_matrix()

,Cns_subregion,Sample_group,Subject_id,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,...,X6B_LINE:CR1:LINE,X7A_LINE:CR1:LINE,X7B_LINE:CR1:LINE,X7C_LINE:CR1:LINE,X7D_LINE:CR1:LINE,X8_LINE:CR1:LINE,X9_LINE:L1:LINE,Zaphod2:hAT-Tip100:DNA,Zaphod3:hAT-Tip100:DNA,Zaphod:hAT-Tip100:DNA
Sample_id,,,,,,,,,,,,,,,,,,,,,
GSM3533230,Frontal Cortex,ALS Spectrum MND,NEUEL133AK6,42,149,0,3327,101,78,4,...,986,2218,2096,1120,84,556,134,570,2045,2446
GSM3533231,Frontal Cortex,ALS Spectrum MND,NEUYV496XLP,25,107,2,1787,153,109,2,...,1216,2373,2001,1144,118,582,153,552,2057,2279
GSM3533232,Frontal Cortex,ALS Spectrum MND,NEUZU200WEQ,20,95,0,1830,121,127,1,...,905,2018,1815,1011,84,571,126,479,1659,1778
GSM3533233,Frontal Cortex,ALS Spectrum MND,NEUME287RK2,22,101,0,2131,142,143,5,...,935,2278,2032,1198,97,562,132,540,2032,2135
GSM3533234,Frontal Cortex,ALS Spectrum MND,NEUEC006FND,22,102,0,1706,83,178,1,...,966,2295,1993,1152,96,582,124,539,1742,1939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM3533401,Frontal Cortex,ALS Spectrum MND,NEUMY028JH4,23,113,0,2130,72,71,2,...,553,1243,1089,658,72,320,73,311,1102,1169
GSM3533402,Frontal Cortex,ALS Spectrum MND,NEUXR145UBL,134,300,0,6077,171,42,8,...,719,1988,1691,966,119,460,109,399,1619,1724
GSM3533403,Frontal Cortex,ALS Spectrum MND,NEUKN209FNW,64,290,0,4752,118,174,8,...,623,1466,1246,726,61,358,107,347,1284,1469


In [63]:
data_by_diseasegrp = c_data.get_data_matrix()['Sample_group'].value_counts()
data_by_diseasegrp

Sample_group
ALS Spectrum MND                145
Non-Neurological Control         17
Other Neurological Disorders     14
Name: count, dtype: int64

In [54]:
data_by_diseasegrp_stats = c_data.get_data_genes().groupby(['Sample_group']).mean()

KeyError: 'Sample_group'

In [30]:
data_by_diseasegrp

Sample_group
ALS Spectrum MND                145
Non-Neurological Control         17
Other Neurological Disorders     14
Name: Sample_id, dtype: int64

In [32]:
data_by_subregion = c_data.get_data_matrix().groupby(['Cns_subregion']).count()['Sample_id']
data_by_subregion

Cns_subregion
Frontal Cortex            81
Motor Cortex               5
Motor Cortex (Lateral)    45
Motor Cortex (Medial)     45
Name: Sample_id, dtype: int64

In [37]:
data_by_subjectid = c_data.get_data_matrix().groupby(['Subject_id']).count()['Sample_id']
data_by_subjectid

Subject_id
GWF14-01     3
JHU 38       1
JHU 73       3
JHU 74       3
JHU 75       1
            ..
PF-UCL-31    1
PF-UCL-46    1
PF-UCL-6     1
PF-UCL-8     1
PF-UCL-9     1
Name: Sample_id, Length: 95, dtype: int64

In [41]:
data_by_subjectid.describe()

count    95.000000
mean      1.852632
std       0.898799
min       1.000000
25%       1.000000
50%       2.000000
75%       3.000000
max       3.000000
Name: Sample_id, dtype: float64